In [1]:
import pynq
import time
from pynq import PL
import xrfclk
import xrfdc
import numpy as np
from fpbinary import FpBinary
import matplotlib.pyplot as plt
from scipy.fftpack import fft, fftfreq, fftshift
import mkidgen3 as g3
import mkidgen3.mkidpynq as mkidpynq
import mkidgen3.daccomb as daccomb
import mkidgen3
import mkidgen3.util
from mkidgen3.daccomb import generate_dac_comb
from mkidgen3.drivers import axiswitch, bintores, capture, ddc, dactable, axififo, rfdc
from mkidgen3.ifboard import IFBoard
import logging
from logging import getLogger
import json

In [ ]:
logging.basicConfig()
logging.getLogger('mkidgen3').setLevel('DEBUG')

In [2]:
ifb = IFBoard()
print(ifb.status())

IFStatus: Powered, boot 1. LO gen3 fractional mode, full calibration. PLL locked.
	Req: 6000 MHz Attained: 6000 MHz Err: 0 MHz
	DAC attens: (1.0, 1.0)
	ADC Attens: (1.0, 1.0)


In [ ]:
ifb.power_on()
print(ifb.status())

In [ ]:
ifb.set_lo(6000, fractional=True, g2_mode=False, full_calibration=True)
ifb.set_attens(output_attens=(2,3), input_attens=(5,None))
print(ifb.status())

In [4]:
st=ifb.status()
st._data

{'global': {'coms': True,
  'boot': 1,
  'ver': 'IFShield v0.5',
  'gen2': False,
  'fract': True,
  'g3fcal': True,
  'lo': 6000,
  'power': True},
 'attens': {'adc1': 5.0, 'adc2': 1.0, 'dac1': 2.0, 'dac2': 3.0},
 'trf': [{'f_REF': 10,
   'f_PFD': 10,
   'f_calclk': 0.3125,
   'df_RF': 5.960465e-07,
   'f_LO': 6000,
   'pll_locked': True},
  {'r0': {'raw': 2326568,
    'chip_id': True,
    'r_sat_err': False,
    'vco_trim': 7,
    'vco_sel': 0,
    'count_mode_mux_sel': False,
    'count': 284}},
  {'r1': {'raw': 1766850601,
    'rdiv': 1,
    'invert_ref_clock': False,
    'neg_vco': True,
    'icp': 10,
    'icpdouble': False,
    'cal_clk_sel': 13}},
  {'r2': {'raw': 209724810,
    'nint': 300,
    'pll_div_sel': 0,
    'prescale_sel': True,
    'vco_sel': 3,
    'vcosel_mode': False,
    'cal_acc': 0,
    'en_cal': False}},
  {'r3': {'raw': 11, 'nfrac': 0}},
  {'r4': {'raw': 1243349004,
    'pwd_pll': False,
    'pwd_cp': False,
    'pwd_vco': False,
    'pwd_vcomux': False,
    

In [ ]:
ol = g3.configure('single_ended/iqtest.bit', clocks=True, external_10mhz=True, ignore_version=True)
print(f"PL Bitfile: {PL.bitfile_name}\nPL Timestamp: {PL.timestamp}\n"
      f"Overlay timestamp: {ol.timestamp}  Loaded: {ol.is_loaded()}")

In [ ]:
#ol.rfdc.select_adc('single_ended')
# ol.rfdc.select_adc('differential')
ol.rfdc.rfdc_status(tell=True);
#print(f'Active ADC pair: {ol.rfdc.active_adc}')

## Verify the cores are all online and won't bring down the board

In [ ]:
ol.capture.filter_iq_0.register_map

In [ ]:
ol.capture.filter_iq_1.register_map

In [ ]:
ol.capture.axis_switch.read(0x0040),ol.capture.axis_switch.is_disabled()

In [ ]:
ol.capture.axis2mm.cmd_ctrl_reg

In [ ]:
ol.capture.filter_phase_0.register_map

## Setup DAC REPLAY

In [ ]:
tones = np.array([0.3e9])
amplitudes = np.array([1.0])

In [ ]:
dactable = g3.set_waveform(tones, fpgen=lambda x: (x*2**15).astype(np.uint16))
qtones = dactable['frequencies']
qphases = dactable['phases']
iqtable = dactable['iq']

#### Or Manually

In [ ]:
tones = np.array([0.3e9])
amplitudes = np.array([1.0])
dactable = generate_dac_comb(frequencies=tones, n_samples=2**19, sample_rate=4.096e9, amplitudes=amplitudes)
#{'iq': iq, 'frequencies': quantized_freqs, 'phases': phases}
qtones = dactable['frequencies']
qphases = dactable['phases']
iqtable = dactable['iq']
buf = np.zeros((2 ** 15, 2, 16), dtype=np.int16)
buf[:, 0, :] = iqtable.real.reshape((2 ** 15,16)) * 2**15
buf[:, 1,: ] = iqtable.imag.reshape((2 ** 15,16)) * 2**15
ol.dac_table_axim_0.stop()
ol.dac_table_axim_0.replay(iqtable, fpgen=lambda x: (x*2**15).astype(np.uint16))
ol.dac_table_axim_0._buffer

In [ ]:
dactable = generate_dac_comb(frequencies=tones, n_samples=2**19, sample_rate=4.096e9, amplitudes=amplitudes)
#{'iq': iq, 'frequencies': quantized_freqs, 'phases': phases}
qtones = dactable['frequencies']
qphases = dactable['phases']
iqtable = dactable['iq']

In [ ]:
buf = np.zeros((2 ** 15, 2, 16), dtype=np.int16)
buf[:, 0, :] = iqtable.real.reshape((2 ** 15,16)) * 2**15
buf[:, 1,: ] = iqtable.imag.reshape((2 ** 15,16)) * 2**15

In [ ]:
ol.rfdc.rfdc.IPStatus, ol.rfdc.rfdc.dac_tiles[1].PLLConfig, ol.rfdc.rfdc.dac_tiles[1].blocks[2].BlockStatus, ol.rfdc.rfdc.dac_tiles[1].blocks[3].BlockStatus

In [ ]:
ol.dac_table_axim_0.stop()
ol.dac_table_axim_0.replay(iqtable, fpgen=lambda x: (x*2**15).astype(np.uint16))

In [ ]:
ol.dac_table_axim_0._buffer

### Zero PL DDR4 if desired

In [ ]:
ddr4_mmio=pynq.MMIO(g3.mkidpynq.PL_DDR4_ADDR, length=2**32)

In [ ]:
ddr4_mmio.array[:]=0

## Resonator selection

In [ ]:
g3.set_channels(tones)
ol.photon_pipe.reschan.bin_to_res.bins

#### Or manually

In [ ]:
bins = np.arange(2048, dtype=int)
bins[:tones.size] = opfb_bin_number(tones)
ol.photon_pipe.reschan.bin_to_res.bins=bins

## DDC Configuration

In [ ]:
g3.set_tones(tones)
ol.photon_pipe.reschan.resonator_ddc.tones

#### Or manually

In [ ]:
ddctones = np.zeros((2, 2048))
ddctones[0, :min(tones.size, 2048)] = tone_increments(tones)
ddctones[1, :] = np.zeros(2048)  #Phase offsets to 0
print('Writing tones...')  # The core expects normalized increments
ol.photon_pipe.reschan.resonator_ddc.tones = tones

## ADC Capture

In [ ]:
raw=ol.capture.capture_adc(2048*32)

In [ ]:
#ol.capture.axis2mm.cmd_ctrl_regd_ctrl_reg

In [ ]:
#raw.shape

In [ ]:
rf=raw.ravel()

In [ ]:
n=128
plt.figure(figsize=(12,8))
plt.axhline(0, color='k')
plt.plot(np.arange(n), rf.astype(np.int16)[:n],'o')
plt.plot(np.arange(n)[::2], rf.astype(np.int16)[:n:2],'^')
# plt.plot(np.arange(n)[::2], -rf.astype(np.int16)[:n:2],'^')
# plt.plot(np.arange(n)[1::2], rf.astype(np.int16)[1:n:2],'.')

In [ ]:
np.arange(20)[::3]

In [ ]:
adcd =raw[:,0]/2**15 + 1j*raw[:,1]/2**15

# Plot Spectrum

In [ ]:
N = 2**16
Fs = 4.096e9 # ADC Sample Rate [Hz]
Tc = N/Fs # total collection time (seconds)
time = np.linspace(0,Tc,N) # Time vector (seconds)
sample = np.linspace(0,N-1,N)
plot_start=10
plot_stop=2047+10
sl = slice(plot_start,plot_stop)
fft_freqs = np.linspace(-2.048e9,2.048e9,plot_stop-plot_start)

In [ ]:
ol.dac_table_axim_0.register_map

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(sample[sl],adcd.real[sl])
plt.plot(sample[sl],adcd.imag[sl])
plt.title('Sample Series')
plt.xlabel("sample",position=(0.5,1))
plt.ylabel("signal",position=(0,0.5))
#ax.set_xlim(time[plot_start],time[plot_stop])
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(sample[sl],np.diff(adcd.imag[10:3151]))
plt.title('Derivative')
plt.xlabel("sample",position=(0.5,1))
plt.ylabel("signal",position=(0,0.5))
#ax.set_xlim(time[plot_start],time[plot_stop])
plt.show()

In [ ]:
a = sample[sl]
b = np.diff(adcd.real[1010:3151])
out = np.column_stack((a, b))

In [ ]:
plt.figure(figsize=(10,5))
# plt.plot(time[sl],(ps_buf&0xffff)[sl].astype(np.int16), '-', label='real')
plt.plot(time[sl],adc_capture_data.real[sl])
plt.grid(True)
plt.xlabel("time(s)",position=(0.5,1))
plt.ylabel("signal(V)",position=(0,0.5))
ax = plt.gca()
ax.set_xlim(time[plot_start],time[plot_stop])
#plt.legend()
plt.title('Time Series')
plt.show()

In [ ]:
plt.plot(np.arange(100),adcd.real[-100:])
plt.plot(np.arange(100)+100,adcd.real[:100])

In [ ]:
plt.plot((np.fft.fftfreq(adcd.size, d=1/4096)), (20*np.log10(np.abs(np.fft.fft(adcd)))))
plt.xlabel('Freq (MHz)');
#plt.xlim(0,500)

In [ ]:
y_fft = 20*np.log10(np.abs(np.fft.fft(adcd)))
plt.plot(np.fft.fftshift(np.fft.fftfreq(y_fft.size, d=1/4.096)), y_fft)

## IQ Capture

Check which capture locations are supported:

In [ ]:
ol.capture.filter_iq

In [ ]:
iq = ol.capture.capture_iq(8192, groups='all', tap_location='ddciq', duration=False) #rawiq=bin2res #iq=lowpassed  #ddciq=before lowpass (unsupported)

## Phase Capture

In [ ]:
phase = ol.capture.capture_phase(8192, groups='all', duration=False)

In [ ]:
phase.shape

In [ ]:
plt.plot(phase[:,0]/2**13)
plt.plot(phase[:,1]/2**13)
plt.plot(phase[:,2]/2**13)
plt.plot(phase[:,3]/2**13)

## Manual phase capture

In [ ]:
n=100
groups = [0,1,2,3,4]  #0-127
self=ol.capture

In [ ]:
self.switch.set_driver(slave=self.SOURCE_MAP['phase'], commit=True)

In [ ]:
print(self.axis2mm.cmd_ctrl_reg)
self.axis2mm.abort()
self.axis2mm.clear_error()
if not self.axis2mm.ready:
    print("capture core unable not ready, this shouldn't happen")
print(self.axis2mm.cmd_ctrl_reg)

In [ ]:
self.filter_phase.keep = groups
n_groups = self.filter_phase.n_kept

In [ ]:
# each group is 16 phases (32 bytes)
capture_bytes = n * 2 * n_groups*16

try:
    buffer = allocate((n, n_groups*16), dtype='i2', target=self.ddr4_0)
except RuntimeError:
    getLogger(__name__).warning(f'Insufficient space for requested samples.')
    raise RuntimeError('Insufficient free space')
addr = buffer.device_address

datavolume_mb = capture_bytes / 1024 ** 2
datarate_mbps = 32 * 512/4 * n_groups/128   #phases arrive 4@512 so the filter outputs every 4 clocks
captime = datavolume_mb / datarate_mbps

msg = (f"Capturing ~{datavolume_mb:.2f} MB of data @ {datarate_mbps:.1f} MBps. "
       f"ETA {datavolume_mb / datarate_mbps * 1000:.0f} ms")
print(msg)

In [ ]:
self.axis2mm.addr = buffer
self.axis2mm.len = n
self.axis2mm.start(continuous=False, increment=True)
self._capture('phase', capture_bytes, addr)
time.sleep(captime)

### Scratchpad